# Evaluation of the Top classifier

This is the evaluation script of the top classifier in the hierarchy of three classifiers. The top classifier outputs a vector of size 2.

The two indices of the vector are the scores that the classifier has calculated based on one 5-second audio clip.

The first indice (0th) is the score/probability that the audio clip is of the Happy/Angry class.

The second indice (1st) is the score/probability that the audio clip is of the Neutral/Sad class.

### library and package importation

In [1]:
import random
import os
import shutil
import glob
import gc
import sys
import h5py
import time
import datetime
import pickle
import librosa
import warnings
import matplotlib.pyplot as plt

from graphs import draw
import graphs

import pickle

import numpy as np
from numpy import array
import pandas as pd
from pandas.plotting import parallel_coordinates
from pydub import AudioSegment

#imported for testing
import wave
import contextlib

# for outputing file
from scipy.cluster.vq import vq, kmeans, whiten
import scipy.stats.stats as st

from sklearn.cluster import KMeans
from sklearn.datasets import make_classification
from sklearn.datasets.samples_generator import make_blobs
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import normalize
from sklearn.metrics import auc, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn import mixture
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.utils import class_weight

import keras

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.layers import Convolution1D, Dense, MaxPooling1D, Flatten, Add, Dropout, Input, Activation
from keras.layers import TimeDistributed, Bidirectional, LSTM, LeakyReLU
from keras.models import Sequential
from keras.models import load_model
from keras import optimizers, regularizers
from keras.utils import np_utils, to_categorical

from IPython.display import clear_output

# confirm TensorFlow sees the GPU
from tensorflow.python.client import device_lib
# assert 'GPU' in str(device_lib.list_local_devices())

# confirm Keras sees the GPU
from keras import backend
# print(len(backend.tensorflow_backend._get_available_gpus()) > 0)

sample_rate=44100
hop_length = 441  # frame size= 2*hop
segment_length=int(sample_rate*0.2)  #0.2
segment_pad=int(sample_rate*0.02)     #0.02
overlappiong=int(sample_rate*0.1)   #0.1

NumofFeaturetoUse = 272 # this will re-assigned for different classifiers
frame_number = 48

sys.path.insert(1, '..//components//')
import load_feat_directories

# input new indices file here
# indices_filename = 'D://indices_filename.npy'
# indices=np.load(indices_filename)

Using TensorFlow backend.
C:\Users\yg9ca\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\yg9ca\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\yg9ca\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\yg9ca\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
def avail_modules(directory, prefix):
    module_names = []
    for item in os.listdir(directory):
        if prefix in item:
            module_names.append(directory + item)
            i = module_names.index(directory + item)
            print(str(i) + 'th module')
            print(directory + item)
    return module_names

In [3]:
def comprise_vector(path, dist, reverb, noise):
    vec_to_return = []
    for fname in os.listdir(path):
        components = fname.split('_')
        '''
        if dist == 0 and 'deamp' in components: continue
        if reverb == 0 and 'WetDry' in components: continue
        if noise == 0 and 'noise' in components: continue
        '''
        current_vec = np.load(path + fname)
        vec_to_return.append(current_vec)
        
    vec_to_return = np.array(vec_to_return)
    return vec_to_return

In [4]:
def comprise_label(feature_vector, label):
    label_vec_to_ret = []
    length = len(list(feature_vector))
    for index in range(0, length):
        current_label = [label]
        label_vec_to_ret.append(current_label)
    label_vec_to_ret = np.array(label_vec_to_ret)

    return label_vec_to_ret

def float_compatible(input_np):

    x = np.where(input_np >= np.finfo(np.float32).max)
    for index in range(0, len(x[0])):
        x_position = x[0][index]
        y_position = x[1][index]
        input_np[x_position, y_position] = 0.0
    input_np = np.nan_to_num(input_np)

    return input_np

def update_progress(progress):
    bar_length = 100
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))
    clear_output(wait = True)
    
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [5]:
# allnoised_npy[0, 1, 2, 3, 4] ==> H, A, N, S, O
# homenoised_npy[0, 1, 2, 3, 4] ==> H, A, N, S, O
all_noised_npy = load_feat_directories.allnoised_npy
all_noised_npy_test = load_feat_directories.allnoised_npy_test
home_noised_npy = load_feat_directories.homenoised_npy
home_noised_npy_test = load_feat_directories.homenoised_npy_test

for index in range(0, 5):
    if not os.path.exists(all_noised_npy_test[index]):
        print(all_noised_npy_test[index] + ' does not exist. Breaking the loop... ')
    if not os.path.exists(home_noised_npy[index]):
        print(home_noised_npy_test[index] + 'does not exist. Breaking the loop... ')

//Users//yegao//Documents//Datasets//padded_deamplified_allnoised_reverberated//npy//Happy_npy_test// does not exist. Breaking the loop... 
//Users//yegao//Documents//Datasets//padded_deamplified_homenoised_reverberated//npy//Happy_npy_test//does not exist. Breaking the loop... 
//Users//yegao//Documents//Datasets//padded_deamplified_allnoised_reverberated//npy//Angry_npy_test// does not exist. Breaking the loop... 
//Users//yegao//Documents//Datasets//padded_deamplified_homenoised_reverberated//npy//Angry_npy_test//does not exist. Breaking the loop... 
//Users//yegao//Documents//Datasets//padded_deamplified_allnoised_reverberated//npy//Neutral_npy_test// does not exist. Breaking the loop... 
//Users//yegao//Documents//Datasets//padded_deamplified_homenoised_reverberated//npy//Neutral_npy_test//does not exist. Breaking the loop... 
//Users//yegao//Documents//Datasets//padded_deamplified_allnoised_reverberated//npy//Sad_npy_test// does not exist. Breaking the loop... 
//Users//yegao//Do

# Top (Binary)

In [36]:
def top_eval_original(emotionFolders):
    total = 0
    
    zero_scores = []
    one_scores = []
    
    correct = 0
    incorrect = 0
    
    result = 0
    
    for emotionFolder in emotionFolders:
        if 'Happy' in emotionFolder: val = 0
        if 'Angry' in emotionFolder: val = 0
        if 'Neutral' in emotionFolder: val = 1
        if 'Sad' in emotionFolder: val = 1

        for emotionfile in os.listdir(emotionFolder):
            
            dist = 'deamp_' in emotionfile
            reverb = 'WetDry_' in emotionfile and 'Diffusion_' in emotionfile and 'DecayFactor_' in emotionfile
            noise = 'noise_' in emotionfile
        
            r_n = not dist and reverb and noise 
            r = not dist and reverb and not noise
            n = not dist and not reverb and noise
            original = not dist and not reverb and not noise
            d_r_n = dist and reverb and noise
            d_r = dist and reverb and not noise
            d_n = dist and not reverb and noise
            d = dist and not reverb and not noise 
            
            if emotionfile.endswith('.npy') and not os.path.isdir(emotionfile):
                one_sample = np.load(emotionFolder + emotionfile)
                one_sample = np.array([one_sample])
                
                pred = model.predict(one_sample)
                zero_score, one_score = pred[0]
                zero_scores.append(zero_score)
                one_scores.append(one_score)
                
                #np.save(emotionFolder + 'two//' + emotionfile, np.array([zero_score, one_score]))
                #print(emotionFolder + 'two//' + emotionfile)
                x = load_lr_model.predict(pred)
                
                if val == 0:
                    if x[0] == 0: correct += 1
                    else: incorrect += 1
                else:
                    if x[0] == 1: correct += 1
                    else: incorrect += 1
                        
                total += 1
    
    print('correct = ' + str(correct))
    print('incorrect = ' + str(incorrect))
    print('error rate = ' + str(incorrect/total) )
    print('accuracy = ' + str(correct/total) )
    print('total files = ' + str(total))
                       
    mydict = {'one_scores': one_scores, 'zero_scores': zero_scores, \
             'error_rate': incorrect/total, 'accuracy': correct/total, 'total_files': total}
    return mydict

In [37]:
directory = '..//..//modules//'
module_prefix = 'Top_'
modules = avail_modules(directory, module_prefix)

0th module
..//..//modules//Checkpoint_home_Top_neurons_home_2048_filters_1024_dropout_0.2_epoch_1000_dense_6.hdf5


In [38]:
directory = '..//..//modules//tf//'
module_prefix_tf = 'Top_'
modules_tf = avail_modules(directory, module_prefix_tf)

0th module
..//..//modules//tf//Top_decision_tree.sav


In [39]:
title = modules[0]
model = keras.models.load_model(title)

In [41]:
load_lr_model = pickle.load(open(modules_tf[0], 'rb'))

## Specify the directories that contain the files of emotion

In [42]:
emodb_Happy = 'D://Datasets//EMO-DB//wav//Happy//'
emodb_Angry = 'D://Datasets//EMO-DB//wav//Angry//'
emodb_Neutral = 'D://Datasets//EMO-DB//wav//Neutral//'
emodb_Sad = 'D://Datasets//EMO-DB//wav//Sad//'
emodb_Other = 'D://Datasets//EMO-DB//wav//Other//'

emodb_Happy_npy = 'D://Datasets//EMO-DB//wav//npy//Happy//'
emodb_Angry_npy = 'D://Datasets//EMO-DB//wav//npy//Angry//'
emodb_Neutral_npy = 'D://Datasets//EMO-DB//wav//npy//Neutral//'
emodb_Sad_npy = 'D://Datasets//EMO-DB//wav//npy//Sad//'
emodb_Other_npy = 'D://Datasets//EMO-DB//wav//npy//Other//'

emodb = [emodb_Happy, emodb_Angry, emodb_Neutral, emodb_Sad, emodb_Other]
emodb_npy = [emodb_Happy_npy, emodb_Angry_npy, emodb_Neutral_npy, emodb_Sad_npy, emodb_Other_npy]

## Only evaluate unmodified wav files

### evaluate the set of Happy validation

In [43]:
dictionary_original_h = top_eval_original([emodb_npy[0]])
one_scores_original_h = dictionary_original_h.get('one_scores')
zero_scores_original_h = dictionary_original_h.get('zero_scores')

correct = 71
incorrect = 0
error rate = 0.0
accuracy = 1.0
total files = 71


### evaluate the set of Angry validation

In [47]:
dictionary_original_a = top_eval_original([emodb_npy[1]])
one_scores_original_a = dictionary_original_a.get('one_scores')
zero_scores_original_a = dictionary_original_a.get('zero_scores')

correct = 126
incorrect = 1
error rate = 0.007874015748031496
accuracy = 0.9921259842519685
total files = 127


### evaluate the set of Neutral validation

In [48]:
dictionary_original_n = top_eval_original([emodb_npy[2]])
one_scores_original_n = dictionary_original_n.get('one_scores')
zero_scores_original_n = dictionary_original_n.get('zero_scores')

correct = 20
incorrect = 59
error rate = 0.7468354430379747
accuracy = 0.25316455696202533
total files = 79


### evaluate the set of Sad validation

In [49]:
dictionary_original_s = top_eval_original([emodb_npy[3]])
one_scores_original_s = dictionary_original_s.get('one_scores')
zero_scores_original_s = dictionary_original_s.get('zero_scores')

correct = 41
incorrect = 21
error rate = 0.3387096774193548
accuracy = 0.6612903225806451
total files = 62


### draw the plot (evaluation on only unmodified, original wav files)

In [ ]:
category_names = ['Error Rate', 'Accuracy']
graph_title = 'Evaluation on only unmodified, original wav files'
results = {
    'Happy': [dictionary_original_h.get('error_rate'), dictionary_original_h.get('accuracy')],
    'Angry': [dictionary_original_a.get('error_rate'), dictionary_original_a.get('accuracy')],
    'Neutral': [dictionary_original_n.get('error_rate'), dictionary_original_n.get('accuracy')],
    'Sad': [dictionary_original_s.get('error_rate'), dictionary_original_s.get('accuracy')]
}

draw(results, category_names, graph_title)